In [1]:
import time
import copy
import numpy as np
import pandas as pd
import random
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl

import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
data = pd.read_csv('twee_df.csv')
data

,date_hour,impressions
0,2021-01-17 04:00:00,10
1,2021-01-17 03:00:00,12
2,2021-01-17 03:00:00,8
3,2021-01-17 03:00:00,4
4,2021-01-17 03:00:00,3
...,...,...
5877,2021-03-11 10:00:00,6
5878,2021-03-11 09:00:00,4
5879,2021-03-11 11:00:00,3
5880,2021-03-11 11:00:00,6


In [3]:
for i in range(len(data)):
    data['date_hour'][i] = data['date_hour'][i].split()[0]
data

<ipython-input-3-a2b8ba5a4b69>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date_hour,impressions
0,2021-01-17,10
1,2021-01-17,12
2,2021-01-17,8
3,2021-01-17,4
4,2021-01-17,3
...,...,...
5877,2021-03-11,6
5878,2021-03-11,4
5879,2021-03-11,3
5880,2021-03-11,6


In [4]:
data['date_hour'] = pd.to_datetime(data['date_hour'], format='%Y-%m-%d')
data

,date_hour,impressions
0,2021-01-17,10
1,2021-01-17,12
2,2021-01-17,8
3,2021-01-17,4
4,2021-01-17,3
...,...,...
5877,2021-03-11,6
5878,2021-03-11,4
5879,2021-03-11,3
5880,2021-03-11,6


In [5]:
data = pd.DataFrame(data.groupby('date_hour')['impressions'].agg('sum'), columns=['a', 'impressions'])
data.drop(['a'], axis=1, inplace=True)

In [6]:
data

,impressions
date_hour,
2021-01-07,337
2021-01-08,1802
2021-01-09,901
2021-01-10,962
2021-01-11,1211
2021-01-12,1051
2021-01-13,2113
2021-01-14,2213
2021-01-15,2284


In [7]:
data2 = pd.read_csv('twee_rl2.csv')
data2.set_index('date_hour', inplace=True, drop=True)
data2

,impressions
date_hour,
2021-01-07,487
2021-01-08,906
2021-01-09,441
2021-01-10,1423
2021-01-11,1325
2021-01-12,814
2021-01-13,1395
2021-01-14,331
2021-01-15,770


In [8]:
learning_rate = 0.0001
gamma = 0.9
buffer_limit = 5000
batch_size = 64

In [19]:
#환경 구축
class Environment:
    
    def __init__(self, data1, data2, budget, history_t=30):
        self.data1 = data1 #twee 1
        self.data2 = data2 #twee 2
        self.budget = budget #광고 집행 예산
        self.history_t = history_t #?
        self.reset() #초기화
        
    def reset(self):
        self.t = 0 #현재 state
        self.done = False
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        ad_a = 0
        ad_b = 0
        self.positions.append(self.data1['impressions'][self.t] + self.data2['impressions'][self.t])
        rate_a = self.data1['impressions'][self.t]/self.positions[self.t] * self.budget
        rate_b = self.data2['impressions'][self.t]/self.positions[self.t] * self.budget

        if act == 1: # redistribute
            ad_a = random.randrange(1000, self.budget, 1000) 
            ad_b = self.budget - ad_a
            if self.positions[self.t-1] < self.positions[self.t]:
                reward += 1
            else:
                reward -= 1
        elif act == 2: # evaluate
            if ad_a in range(int(rate_a*0.8), int(rate_a*1.2)):
                reward += 1
            else:
                reward -= 1
            if ad_b in range(int(rate_b*0.8), int(rate_b*1.2)):
                reward += 1
            else:
                reward -= 1
        
        # set next time
        self.t += 1
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data1['impressions'][self.t]+self.data2['impressions'][self.t] - p)
        self.history.pop(0)
        self.history.append(self.data1['impressions'][self.t] + data2['impressions'][self.t] - self.data1['impressions'][self.t-1] + data2['impressions'][self.t-1])
        # self.history.append(self.data1.iloc[self.t, :]['impressions'] - self.data1.iloc[(self.t-1), :]['impressions'])
        
        # clipping reward
        if reward > 0:   
            reward = 1
        elif reward < 0:
            reward -= 1
        return [self.position_value] + self.history, reward, self.done # obs, reward

In [20]:
env = Environment(data, data2, 100000)
print(Environment(data, data2, 100000).reset())
for _ in range(2):
    pact = np.random.randint(2)
    print(Environment(data, data2, 100000).step(pact))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
([1884, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2858], -2, False)
([1884, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2858], -2, False)


In [13]:
#리플레이 버퍼 Class
#최신 5만 개의 데이터를 들고 있다가 필요한 때마다 batch_size 만큼의 데이터를 뽑아서 제공해주는 것
class ReplayBuffer(): 
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    #데터를 버퍼에 넣어주는 함수
    def put(self, transition): 
        self.buffer.append(transition)

    #랜덤하게 32개의 데이터를 뽑아 미니배치를 구성해주는 함수
    #done_mask는 종료 상태의 value를 마스킹해주기 위한 변수(종료 상태의 Value를 0으로 만들어 줌)
    #32개의 데이터는 각기 종류에 맞게 (s는 s끼리) 요소별로 모아서 pytorch의 텐서로 변환해주는 작업을 진행
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)

    def size(self):
        return len(self.buffer)                                   

In [14]:
#Q_value network class
#nn.module 클래스를 상속받아 선언됌 - 뉴럴넷을 만들때 뼈대가 되는 클래스
#Q 네트워크의 파라미터들을 타킷 네트워크로 복사할 때 load_state_dict() 함수를 사용해 한 줄만으로 구현이 가능
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 1)
        else:
            return out.argmax().item() #Q값이 제일 큰 액션을 선택

In [32]:
#학습 함수 - 에피소드 하나가 끝날 때마다 호출
def train(q, q_target, memory, optimizer):
    for i in range(10): #한 번 호출할 때마다 10개의 미니 배치를 뽑아 총 10번 업데이트하도록 구현 (한 에피소드에 총 320개의 데이터를 뽑아 사용)
        s, a, r, s_prime, done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime). max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backword() #역전파 
        optimizer.step()

In [33]:
#main 함수
def main():
    env = Environment(data, data2, 100000)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict()) #q 네트워크의 파라미터의 값들을 q_target으로 전부 복사
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000): #에피소드 진행 수
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200))
        #Decaying epsilon
        s = env.reset()
        done = False

        while not done:
            a = q.sample_action(torch.from_numpy(s). float (), epsilon)
            s_prime, r, done, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0, s_prime, done_mask)) #보상을 적절하게 주기 위해서 100으로 나누어 줌
            s = s_prime
            score += r
            if done:
                break
        
        if memory.size()>2000: #데이터가 많이 쌓이지 않은 초기 상태에서 학습을 진행 시 학습이 치우치는 걸 방지하기 위해 2000개를 넘었을 때부터 학습 진행
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi != 0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode : {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format (n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0

    env.close()

In [34]:
main()

TypeError: expected np.ndarray (got list)